## Langchain-based Youtube Search Agent

The goal of this agent is to search Youtube and find relevant videos given a keyword.
Is this a true agent? Well,the program is capable to discern tool use. Besides that 
The "agent" does not reason, plan, and/or critizise its actions.
Therefore, it does what it's supposed to do, but there is no autonomy or agency.
Hence, it is an agent as it is commonly used in 2024, but it is not an agent in the sense of E21A

In [6]:
#%pip install --upgrade torch openai langchain langchain_core langchain_openai langchain_community langgraph youtube_search

Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch

In [3]:
import os, openai

from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain_openai.chat_models import ChatOpenAI

from langchain_core.runnables import RunnablePassthrough
from langchain_core.agents import AgentFinish

from langchain_community.tools import YouTubeSearchTool

from langgraph.graph import END, Graph


## This agent uses OpenAI's model to work

Hence you need a funded OpenAI account for this to work.

In [ ]:
openai_api_key="<YOUR OPENAI KEY>"

Initialize the agent based on default langchain criteria

In [7]:
def initialize(openai_api_key):

    if not openai_api_key:
        print("Can't initialize. OpenAI key is missing")
        return

    #openai.api_key=key
    os.environ["OPENAI_API_KEY"]=openai_api_key
   
    # Get the prompt to use (default
    prompt = hub.pull("hwchase17/openai-functions-agent")

    # Choose the LLM that will drive the agent
    llm = ChatOpenAI(model="gpt-4", openai_api_key=openai_api_key, temperature=0.2)

    tools = [YouTubeSearchTool()]

    agent_runnable = create_openai_functions_agent(llm, tools, prompt)

    # Define the agent
    agent = RunnablePassthrough.assign(
        agent_outcome = agent_runnable
    )

    # Define the function to execute tools
    def execute_tools(data):
        agent_action = data.pop('agent_outcome')
        tool_to_use = {t.name: t for t in tools}[agent_action.tool]
        observation = tool_to_use.invoke(agent_action.tool_input)
        data['intermediate_steps'].append((agent_action, observation))
        print(data)
        return data

    # Define logic that will be used to determine which conditional edge to go down
    def should_continue(data):
        if isinstance(data['agent_outcome'], AgentFinish):
            return "exit"
        else:
            return "continue"
        
    workflow = Graph()

    workflow.add_node("agent", agent)
    workflow.add_node("tools", execute_tools)

    # Set the entrypoint as `agent`
    workflow.set_entry_point("agent")

    workflow.add_conditional_edges(
        "agent",
        should_continue,
        {
            "continue": "tools",
            "exit": END
        }
    )
    workflow.add_edge('tools', 'agent')

    # This compiles it into a LangChain Runnable,
    # meaning we can use it as you would any other runnable
    chain = workflow.compile()

    return chain

The below segment looks a bit messy. And probably all the "message appends" are not really necessary. 

In [8]:
messages= [{"role": "assistant", "content": "How can I help you?"}]

chain = initialize(openai_api_key)

prompt = "Search for videos containing cute puppies"

messages.append({"role": "user", "content": prompt})

result = chain.invoke({"input": prompt, "intermediate_steps": []})
    
msg = result['agent_outcome'].return_values["output"]

print(msg)

messages.append({"role": "assistant", "content": msg})


/home/moebius/Projects/HF_Data/venv/lib/python3.12/site-packages/langsmith/client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


{'input': 'Search for videos containing cute puppies', 'intermediate_steps': [(AgentActionMessageLog(tool='youtube_search', tool_input={'query': 'cute puppies', 'run_manager': <langchain_core.callbacks.manager.CallbackManagerForToolRun object at 0x7a5f2402c8f0>}, log="\nInvoking: `youtube_search` with `{'query': 'cute puppies'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "query": "cute puppies"\n}', 'name': 'youtube_search'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 104, 'total_tokens': 121, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-0783a4c3-fca5-49b5-b70f-9c782e26070a-0', usage_metadata={'input_tokens': 104, 'output_tokens': 17, 'total_tokens': 121})]), "['https://www.youtube.com/watch?v=VAH-ixdFWFs&pp=ygUMY3V0ZSBwdXBwaWVz', 'https://www.youtube.com